# Рекомендация тарифов

Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».


В нашем распоряжении данные о поведении клиентов, которые уже перешли на тарифы. Нужно построить модель для задачи классификации, которая выберет подходящий тариф.

Построим модель с максимально большим значением *accuracy*. Цель - 0.75. Проверим *accuracy* на тестовой выборке.

## Откроем и изучим файл

Импортируем необходимые библиотеки

In [1]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import random

Откроем файл с данными

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')

Описание данных

Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:
- сalls — количество звонков,
- minutes — суммарная длительность звонков в минутах,
- messages — количество sms-сообщений,
- mb_used — израсходованный интернет-трафик в Мб,
- is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

In [3]:
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Предобработка данных не требуется, пропущенных значений нет.

Посмотрим на соотношение между классами

In [5]:
df.is_ultra.value_counts()/df.shape[0]

0    0.693528
1    0.306472
Name: is_ultra, dtype: float64

## Разобьем данные на выборки

Извлекем признаки для обучения:

In [6]:
features = df.drop(['is_ultra'], axis=1)
target = df['is_ultra']

Выделим 25% данных для тестовой выборки, остальные — для валидационной.

In [7]:
features_train, features_test_valid, target_train, target_test_valid = (train_test_split(features,
                                                                               target, 
                                                                               test_size=0.40, 
                                                                               random_state=12345) 
                                                             )

In [8]:
features_test, features_valid, target_test, target_valid = (train_test_split(features_test_valid,
                                                                               target_test_valid, 
                                                                               test_size=0.50, 
                                                                               random_state=12345) 
                                                             )

## Исследуем модели

### Модель дерево решений

Проверим точность модели на валидационной выборке и найдем оптимальные гиперпараметры

In [9]:
%%time
1 + 1
best_model = None
best_result = 0
best_depth = 0
for depth in range(1, 11):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)# инициализируем модель DecisionTreeRegressor
    model.fit(features_train, target_train) # обучим модель на тренировочной выборке
    predictions_valid = model.predict(features_valid) # получим предсказания модели на валидационной выборке

    result = accuracy_score(target_valid, predictions_valid)
    if result > best_result:
        best_model = model
        best_result = result
        best_depth = depth
print("accuracy наилучшей модели на валидационной выборке:", best_result, "Глубина дерева:", best_depth)

accuracy наилучшей модели на валидационной выборке: 0.7993779160186625 Глубина дерева: 7
CPU times: user 83.6 ms, sys: 5.33 ms, total: 88.9 ms
Wall time: 106 ms


### Модель случайный лес

In [10]:
%%time
1 + 1
best_model = None
best_result = 0
best_depth = 0
for est in range(1, 11):
    for depth in range(1,11):
        model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth = depth)
        model.fit(features_train, target_train) 
        result = model.score(features_valid, target_valid)
        if result > best_result:
            best_model = model
            best_result = result
            best_est = est
            best_depth = depth
print("accuracy наилучшей модели на валидационной выборке:", best_result, "Количество деревьев:", best_est,  "Глубина дерева:", best_depth)

accuracy наилучшей модели на валидационной выборке: 0.8133748055987559 Количество деревьев: 8 Глубина дерева: 9
CPU times: user 2.3 s, sys: 33.1 ms, total: 2.33 s
Wall time: 2.4 s


### Модель логистическая регрессия

In [11]:
%%time
1 + 1
best_model = None
best_result = 0
best_depth = 0
model = LogisticRegression(random_state=12345) 
model.fit(features_train, target_train)
result = model.score(features_valid, target_valid)
print("accuracy наилучшей модели на валидационной выборке:", result)

accuracy наилучшей модели на валидационной выборке: 0.6842923794712286
CPU times: user 25.9 ms, sys: 12.6 ms, total: 38.4 ms
Wall time: 115 ms


**Вывод:** Видим что у случайного леса и решающего дерева одинаково высокое качество, у дерева решений быстродействие выше. Логистическая регрессия имеет наиболее высокое быстродействие и низкую точность.

## Проверим модель на тестовой выборке

Проверим модель показавшую лучший результат на валидационной выборке - дерево решений на тестовой выборке

In [12]:
%%time
1 + 1
model = DecisionTreeClassifier(random_state=12345, max_depth=7)
model.fit(features_train, target_train) 
predictions_test = model.predict(features_test) 
result = accuracy_score(target_test, predictions_test)
print("accuracy модели на тестовой выборке:", result)

accuracy модели на тестовой выборке: 0.7822706065318819
CPU times: user 16.1 ms, sys: 3.92 ms, total: 20 ms
Wall time: 20.7 ms


Результат качества выше целевого 0,75.

## Проверим модели на адекватность

Чтобы оценить адекватность модели в задачах классификации, нужно сравнить её со случайной. Ответим на все объекты одним и тем же классом. Чтобы оно не сильно расходилось с правдой, примем за этот класс наиболее популярный тариф.

Проверим соотношение количества клиентов с тарифом Ультра и Смарт

In [13]:
df.is_ultra.mean()

0.30647168637212197

30% клиентов пользуются тарифом ультра. Остальные тарифом смарт.

In [14]:
predictions_mean = pd.Series(0, index=target.index)

In [15]:
result = accuracy_score(target, predictions_mean)
print("accuracy модели при предсказании одного и того же тарифа:", result)

accuracy модели при предсказании одного и того же тарифа: 0.693528313627878


**Вывод:** точность полученная с использованием модели выше, чем при случайных предсказанных значениях. Целесообранее использовать модель для предложения пользователям нового тарифа.